In [1]:
# Video segmentation
import cv2
import os
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
import math

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def split_video(input_video_path, output_dir, segment_duration, gap_duration, new_fps=None):
    """Split a single video file into segments with specified duration and gap.
       Simplified strategy:
       - Extract segment_frames frames from original video based on original fps
       - If new_fps < original fps: subsample frames to achieve lower frame rate
       - If new_fps > original fps: pad with last frame to maintain duration
    """
    try:
        # Check if input video exists
        if not os.path.exists(input_video_path):
            logging.error(f"Video file not found: {input_video_path}")
            return

        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        logging.info(f"Processing video: {input_video_path}")
        cap = cv2.VideoCapture(input_video_path)
        if not cap.isOpened():
            logging.error(f"Failed to open video file: {input_video_path}")
            return

        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Use original fps if new_fps is not specified
        output_fps = new_fps if new_fps is not None else fps

        # Calculate segment and gap frames based on original fps
        segment_frames = int(segment_duration * fps)
        gap_frames = int(gap_duration * fps)

        # Calculate required frames for output segment to maintain duration
        segment_output_frames = int(segment_duration * output_fps)

        base_filename = os.path.splitext(os.path.basename(input_video_path))[0]

        start_frame = 0
        segment_index = 1
        
        # Process video in segments
        while start_frame + segment_frames <= total_frames:
            # Position to start of segment
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            original_segment = []
            
            # Read segment frames
            for _ in range(segment_frames):
                ret, frame = cap.read()
                if not ret:
                    break
                original_segment.append(frame)

            original_segment_length = len(original_segment)
            if original_segment_length == 0:
                break

            # Resample frames according to new_fps
            sampled_frames = []
            if new_fps is None or abs(new_fps - fps) < 1e-9:
                # No fps change specified or nearly same as original
                # Adjust frame count by truncating or padding
                if original_segment_length >= segment_output_frames:
                    # Take first segment_output_frames frames if we have enough
                    sampled_frames = original_segment[:segment_output_frames]
                else:
                    # Pad with last frame if we don't have enough frames
                    sampled_frames = original_segment + [original_segment[-1]]*(segment_output_frames - original_segment_length)
            else:
                # FPS change specified
                # For new_fps <= fps: uniformly sample segment_output_frames frames
                ratio = original_segment_length / segment_output_frames
                for j in range(segment_output_frames):
                    pos = j * ratio
                    idx = int(math.floor(pos))
                    if idx >= original_segment_length:
                        idx = original_segment_length - 1
                    sampled_frames.append(original_segment[idx])

                # For new_fps > fps: pad with last frame if needed
                if len(sampled_frames) < segment_output_frames:
                    sampled_frames += [sampled_frames[-1]] * (segment_output_frames - len(sampled_frames))

            # Write output segment
            segment_output_path = os.path.join(output_dir, f"{base_filename}_segment_{segment_index}.mp4")

            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(segment_output_path, fourcc, output_fps,
                                (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                                int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

            for frame in sampled_frames:
                out.write(frame)

            out.release()
            logging.info(f"Segment {segment_index} saved to: {segment_output_path}")

            segment_index += 1
            start_frame += segment_frames + gap_frames

        cap.release()
        logging.info(f"Video processing complete. Segments saved to: {output_dir}")

    except Exception as e:
        logging.error(f"Error processing video {input_video_path}: {e}")

def split_all_videos_in_folder(input_folder, output_folder, segment_duration, gap_duration, max_workers=4, new_fps=None):
    """Process all videos in a folder, splitting them into segments with optional fps conversion"""
    try:
        # Find all video files in input folder
        video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

        if not video_files:
            logging.info("No video files found in folder to process")
            return

        logging.info(f"Found {len(video_files)} video files, starting processing...")

        # Process videos in parallel
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for video_file in video_files:
                input_video_path = os.path.join(input_folder, video_file)
                output_dir = os.path.join(output_folder, os.path.splitext(video_file)[0])
                futures.append(executor.submit(
                    split_video, input_video_path, output_dir, 
                    segment_duration, gap_duration, new_fps=new_fps))

            # Handle completion/failure of each task
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    logging.error(f"Error processing video: {e}")

        logging.info("All videos processed successfully.")

    except Exception as e:
        logging.error(f"Error processing video folder: {e}")

In [2]:
# Video segmentation
if __name__ == "__main__":
    # Input folder containing original videos
    input_folder = "./raw_video"
    
    # Output folder for segmented videos
    output_folder = "./segment_video"
    
    # Duration parameters (in seconds)
    segment_duration = 30  # Length of each video segment
    gap_duration = 60      # Time gap between segments
    
    # Parallel processing settings
    max_workers = 4        # Number of parallel threads
    
    # Video output settings
    new_fps = 10           # Target frame rate for output videos
    
    # Process all videos in input folder
    split_all_videos_in_folder(
        input_folder=input_folder,
        output_folder=output_folder,
        segment_duration=segment_duration,
        gap_duration=gap_duration,
        max_workers=max_workers,
        new_fps=new_fps
    )

2025-04-19 11:08:14,198 - Found 1 video files, starting processing...
2025-04-19 11:08:14,202 - Processing video: ./raw_video\oakisland_east-2024-08-13-231537Z.mp4
2025-04-19 11:08:35,640 - Segment 1 saved to: ./segment_video\oakisland_east-2024-08-13-231537Z\oakisland_east-2024-08-13-231537Z_segment_1.mp4
2025-04-19 11:08:58,427 - Segment 2 saved to: ./segment_video\oakisland_east-2024-08-13-231537Z\oakisland_east-2024-08-13-231537Z_segment_2.mp4
2025-04-19 11:09:15,364 - Segment 3 saved to: ./segment_video\oakisland_east-2024-08-13-231537Z\oakisland_east-2024-08-13-231537Z_segment_3.mp4
2025-04-19 11:09:30,907 - Segment 4 saved to: ./segment_video\oakisland_east-2024-08-13-231537Z\oakisland_east-2024-08-13-231537Z_segment_4.mp4
2025-04-19 11:09:43,804 - Segment 5 saved to: ./segment_video\oakisland_east-2024-08-13-231537Z\oakisland_east-2024-08-13-231537Z_segment_5.mp4
2025-04-19 11:09:57,162 - Segment 6 saved to: ./segment_video\oakisland_east-2024-08-13-231537Z\oakisland_east-2024-

In [3]:
# Video cropping with custom selection boxes
import cv2
import os
import json
import logging

# Input/output paths configuration
input_folder = './segment_video/oakisland_east-2024-08-13-231537Z'  # Path to input video folder
output_folder = './clip_video'  # Path for output cropped videos
config_file = os.path.join(output_folder, 'video_boxes.json')  # JSON file to store box coordinates
progress_file = os.path.join(output_folder, 'video_progress.json')  # File to track processing progress

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get all video files in input folder (MP4 format)
video_files = [f for f in os.listdir(input_folder) if f.endswith('.mp4')]

# Fixed selection box size
search_box_size = (224, 224)  # Fixed size for selection boxes (width, height)
search_boxes = {}  # Dictionary to store box coordinates for each video
removed_boxes = {}  # Dictionary to store undone box selections
dragging = False  # Flag to track if box selection is in progress
start_point = None  # Starting coordinates for box selection

# Load previously saved box selections if config file exists
if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        search_boxes = json.load(f)

# Load processing progress to resume from last position
processed_videos = []
if os.path.exists(progress_file):
    with open(progress_file, 'r') as f:
        processed_videos = json.load(f)

def select_box(event, x, y, flags, param):
    """Mouse callback function for box selection"""
    global start_point, dragging, frame, search_box_size, search_boxes, removed_boxes, current_video_file

    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button pressed - start selection
        start_point = (x, y)
        dragging = True

    elif event == cv2.EVENT_MOUSEMOVE:  # Mouse moved - update selection box
        if dragging:
            temp_frame = frame.copy()  # Create temporary frame for drawing
            end_point = (x + search_box_size[0], y + search_box_size[1])
            cv2.rectangle(temp_frame, (x, y), end_point, (0, 255, 0), 2)
            
            # Draw all selected boxes (green)
            for box in search_boxes.get(current_video_file, []):
                cv2.rectangle(temp_frame, box[0], box[1], (0, 255, 0), 2)
            
            # Draw all undone boxes (red)
            for box in removed_boxes.get(current_video_file, []):
                cv2.rectangle(temp_frame, box[0], box[1], (0, 0, 255), 2)
            
            cv2.imshow("Select Region", temp_frame)

    elif event == cv2.EVENT_LBUTTONUP:  # Left mouse button released - finalize selection
        dragging = False
        end_point = (x + search_box_size[0], y + search_box_size[1])
        if current_video_file not in search_boxes:
            search_boxes[current_video_file] = []
        search_boxes[current_video_file].append(((x, y), (end_point[0], end_point[1])))
        cv2.rectangle(frame, (x, y), (end_point[0], end_point[1]), (0, 255, 0), 2)
        cv2.imshow("Select Region", frame)
        print(f"Added box: {((x, y), (end_point[0], end_point[1]))}")

def undo_last_selection():
    """Undo the most recent box selection"""
    global current_video_file
    if current_video_file in search_boxes and len(search_boxes[current_video_file]) > 0:
        removed_boxes.setdefault(current_video_file, []).append(search_boxes[current_video_file].pop())
        print("Undid last box selection")

# Flag to control early exit from selection process
exit_flag = False

# Process each video file for box selection
for idx, video_file in enumerate(video_files):
    # Skip already processed videos
    if video_file in processed_videos:
        print(f"Skipping already processed video: {video_file}")
        continue

    input_video_path = os.path.join(input_folder, video_file)
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Set current video file name
    current_video_file = video_file

    # Skip if exit flag is set
    if exit_flag:
        print(f"Skipping video: {video_file}")
        continue

    # Initialize box data for current video
    if current_video_file not in search_boxes:
        search_boxes[current_video_file] = []

    # Create window for box selection
    cv2.namedWindow("Select Region", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Select Region", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.setMouseCallback("Select Region", select_box)

    # Box selection loop
    while True:
        ret, frame = cap.read()
        if not ret:
            # Loop video from beginning when finished
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            continue

        # Create temporary frame for drawing
        temp_frame = frame.copy()

        # Draw all selected boxes (green)
        for box in search_boxes.get(current_video_file, []):
            cv2.rectangle(temp_frame, box[0], box[1], (0, 255, 0), 2)

        # Draw all undone boxes (red)
        for box in removed_boxes.get(current_video_file, []):
            cv2.rectangle(temp_frame, box[0], box[1], (0, 0, 255), 2)

        # Draw current selection box (green)
        if dragging and start_point:
            end_point = (start_point[0] + search_box_size[0], start_point[1] + search_box_size[1])
            cv2.rectangle(temp_frame, start_point, end_point, (0, 255, 0), 2)

        cv2.imshow("Select Region", temp_frame)

        # Handle keyboard input
        key = cv2.waitKey(1)
        if key == 32:  # Space - confirm selection
            break
        elif key == ord('z'):  # 'z' - undo last selection
            undo_last_selection()
        elif key == 27:  # ESC - exit selection process
            print("Exiting box selection for all videos")
            exit_flag = True
            break

    # Clean up window
    cv2.destroyAllWindows()

    # Skip if no regions were selected
    if len(search_boxes[current_video_file]) == 0:
        print(f"{video_file} - no regions selected, skipping")
        cap.release()
        continue

    # Reopen video for processing
    cap.release()
    cap = cv2.VideoCapture(input_video_path)

    print(f"Completed box selection for {video_file} ({idx+1}/{len(video_files)})")

    # Save box selections to JSON file
    with open(config_file, 'w') as f:
        json.dump(search_boxes, f)

    # Update progress tracking
    processed_videos.append(video_file)
    with open(progress_file, 'w') as f:
        json.dump(processed_videos, f)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

def process_videos():
    """Process videos using selected boxes to create cropped clips"""
    for video_file, boxes in search_boxes.items():
        input_video_path = os.path.join(input_folder, video_file)

        # Create output subfolder for current video
        video_output_folder = os.path.join(output_folder, video_file.split('.')[0])
        if not os.path.exists(video_output_folder):
            os.makedirs(video_output_folder)

        # Get video properties
        cap_test = cv2.VideoCapture(input_video_path)
        if not cap_test.isOpened():
            logging.error(f"Failed to open video file: {input_video_path}")
            continue
        fps = cap_test.get(cv2.CAP_PROP_FPS)
        frame_width = int(cap_test.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap_test.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap_test.release()

        # Process each selected box
        for idx, box in enumerate(boxes):
            x1, y1 = box[0]
            x2, y2 = box[1]

            # Validate box coordinates
            if x2 <= x1 or y2 <= y1:
                print(f"Invalid box coordinates, skipping: ({x1}, {y1}), ({x2}, {y2})")
                continue

            # Set output path for cropped clip
            output_video_path = os.path.join(video_output_folder, f"{video_file.split('.')[0]}_clip_{idx+1:03d}.mp4")

            # Skip if output file already exists
            if os.path.exists(output_video_path):
                print(f"File {output_video_path} already exists, skipping")
                continue

            # Process video for current box
            cap = cv2.VideoCapture(input_video_path)
            if not cap.isOpened():
                logging.error(f"Failed to reopen video file: {input_video_path}")
                continue

            # Initialize video writer
            out = cv2.VideoWriter(output_video_path,
                                cv2.VideoWriter_fourcc(*'mp4v'),
                                fps,
                                (x2 - x1, y2 - y1))

            # Process each frame
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Crop frame to selected region
                cropped_frame = frame[y1:y2, x1:x2]
                out.write(cropped_frame)

            # Clean up
            out.release()
            cap.release()

            logging.info(f"Cropped clip {idx+1} saved to: {output_video_path}")

# Execute video processing
process_videos()

print("All video processing completed")


Added box: ((1051, 804), (1275, 1028))
Added box: ((2077, 860), (2301, 1084))
Completed box selection for oakisland_east-2024-08-13-231537Z_segment_1.mp4 (1/7)
Added box: ((1212, 751), (1436, 975))
Added box: ((1889, 798), (2113, 1022))
Completed box selection for oakisland_east-2024-08-13-231537Z_segment_2.mp4 (2/7)
Added box: ((999, 688), (1223, 912))
Added box: ((1860, 823), (2084, 1047))
Added box: ((1464, 712), (1688, 936))
Completed box selection for oakisland_east-2024-08-13-231537Z_segment_3.mp4 (3/7)
Added box: ((922, 712), (1146, 936))
Added box: ((1478, 709), (1702, 933))
Added box: ((1895, 735), (2119, 959))
Completed box selection for oakisland_east-2024-08-13-231537Z_segment_4.mp4 (4/7)
Added box: ((1084, 763), (1308, 987))
Added box: ((1590, 755), (1814, 979))
Added box: ((1968, 712), (2192, 936))
Completed box selection for oakisland_east-2024-08-13-231537Z_segment_5.mp4 (5/7)
Added box: ((782, 709), (1006, 933))
Added box: ((1356, 784), (1580, 1008))
Added box: ((2113,

2025-04-19 11:11:49,313 - Cropped clip 1 saved to: ./clip_video\oakisland_east-2024-08-13-231537Z_segment_1\oakisland_east-2024-08-13-231537Z_segment_1_clip_001.mp4
2025-04-19 11:11:51,901 - Cropped clip 2 saved to: ./clip_video\oakisland_east-2024-08-13-231537Z_segment_1\oakisland_east-2024-08-13-231537Z_segment_1_clip_002.mp4
2025-04-19 11:11:55,276 - Cropped clip 1 saved to: ./clip_video\oakisland_east-2024-08-13-231537Z_segment_2\oakisland_east-2024-08-13-231537Z_segment_2_clip_001.mp4
2025-04-19 11:11:58,044 - Cropped clip 2 saved to: ./clip_video\oakisland_east-2024-08-13-231537Z_segment_2\oakisland_east-2024-08-13-231537Z_segment_2_clip_002.mp4
2025-04-19 11:12:01,088 - Cropped clip 1 saved to: ./clip_video\oakisland_east-2024-08-13-231537Z_segment_3\oakisland_east-2024-08-13-231537Z_segment_3_clip_001.mp4
2025-04-19 11:12:03,836 - Cropped clip 2 saved to: ./clip_video\oakisland_east-2024-08-13-231537Z_segment_3\oakisland_east-2024-08-13-231537Z_segment_3_clip_002.mp4
2025-04-19

All video processing completed


In [9]:
# Frame extraction
def extract_frames_from_videos(input_folder, output_folder, frame_rate=1):
    """
    Extract frames from videos in the specified folder and save results to output path.
    
    Parameters:
    - input_folder (str): Path to folder containing video files to process
    - output_folder (str): Path to save extracted frames
    - frame_rate (int): Number of frames to extract per second
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process all video files in input folder
    for root, _, files in os.walk(input_folder):
        for file in files:
            # Filter for video files (common extensions)
            if file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                video_path = os.path.join(root, file)
                video_name = os.path.splitext(file)[0]
                
                # Create subfolder for this video's frames
                video_output_folder = os.path.join(output_folder, video_name)
                if not os.path.exists(video_output_folder):
                    os.makedirs(video_output_folder)

                # Open video file
                cap = cv2.VideoCapture(video_path)
                if not cap.isOpened():
                    print(f"Failed to open video file: {video_path}")
                    continue
                
                # Calculate frame interval based on desired frame rate
                fps = cap.get(cv2.CAP_PROP_FPS)
                frame_interval = int(fps // frame_rate)

                frame_count = 0
                saved_frame_count = 0

                # Process each frame
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # Save frame at specified interval
                    if frame_count % frame_interval == 0:
                        frame_filename = os.path.join(
                            video_output_folder, f"frame_{saved_frame_count:06d}.jpg"
                        )
                        cv2.imwrite(frame_filename, frame)
                        saved_frame_count += 1

                    frame_count += 1

                cap.release()
                print(f"Video {video_name} processed, extracted {saved_frame_count} frames.")

    print("All videos processed successfully!")

In [12]:
# Frame Extraction Example

input_folder = "./clip_video/oakisland_east-2024-08-13-231537Z_segment_1"  # Path to input video folder
output_folder = "./frame"  # Output directory for extracted frames
frame_rate = 2  # Number of frames to extract per second

# Execute frame extraction
extract_frames_from_videos(input_folder, output_folder, frame_rate)

Video oakisland_east-2024-08-13-231537Z_segment_1_clip_001 processed, extracted 60 frames.
Video oakisland_east-2024-08-13-231537Z_segment_1_clip_002 processed, extracted 60 frames.
All videos processed successfully!
